[![ebac_logo-data_science.png](https://raw.githubusercontent.com/rhatiro/Curso_EBAC-Profissao_Cientista_de_Dados/main/ebac-course-utils/media/logo/ebac_logo-data_science.png)](https://github.com/rhatiro/Curso_EBAC-Profissao_Cientista_de_Dados)
<!-- <img src="https://raw.githubusercontent.com/rhatiro/Curso_EBAC-Profissao_Cientista_de_Dados/main/ebac-course-utils/media/logo/ebac_logo-data_science.png" alt="ebac_logo-data_science"> -->

---

<!-- # **Profissão: Cientista de Dados** -->
### **Módulo 21** | TdD V - SQL | Exercício 3

**Aluno:** [Roberto Hatiro Nishiyama](https://www.linkedin.com/in/rhatiro/)<br>
**Data:** 24 de maio de 2023.

---

## 0. Import das bibliotecas

In [1]:
import psycopg2 as pg2
import pandas   as pd

## 1. Calcule a **média** por nome e sobrenome do ator da seguintes variáveis:
  - `rental_duration`
  - `rental_rate`
  - `length`
  - `replacement_cost`

> **SQL Query:**

```sql
SELECT first_name,
    last_name,
    ROUND(AVG(rental_duration), 2) AS avg_rental_duration,
    ROUND(AVG(rental_rate), 2) AS avg_rental_rate,
    ROUND(AVG(length), 2) AS avg_length,
    ROUND(AVG(replacement_cost), 2) AS avg_replacement_cost
FROM film
LEFT JOIN film_actor
    ON film.film_id = film_actor.film_id
LEFT JOIN actor
    ON film_actor.actor_id = actor.actor_id
WHERE first_name != 'na'
GROUP BY first_name, last_name
ORDER BY first_name, last_name;
```

In [2]:
conn = pg2.connect(host='localhost', 
                   port='5432', 
                   dbname='dvdrental', 
                   user='postgres', 
                   password='123')
cur = conn.cursor()

query = '''
        SELECT first_name,
            last_name,
            ROUND(AVG(rental_duration), 2) AS avg_rental_duration,
            ROUND(AVG(rental_rate), 2) AS avg_rental_rate,
            ROUND(AVG(length), 2) AS avg_length,
            ROUND(AVG(replacement_cost), 2) AS avg_replacement_cost
        FROM film
        LEFT JOIN film_actor
            ON film.film_id = film_actor.film_id
        LEFT JOIN actor
            ON film_actor.actor_id = actor.actor_id
        WHERE first_name != 'na'
        GROUP BY first_name, last_name
        ORDER BY first_name, last_name;
        '''
cur.execute(query)
conn.commit()

data = cur.fetchall()

cur.close()
conn.close()

col_names = [desc.name for desc in cur.description]
pd.DataFrame(data=data, columns=col_names)

,first_name,last_name,avg_rental_duration,avg_rental_rate,avg_length,avg_replacement_cost
0,Adam,Grant,5.28,2.77,122.39,22.93
1,Adam,Hopper,4.86,2.81,112.27,19.49
2,Al,Garland,5.12,2.76,106.38,19.87
3,Alan,Dreyfuss,5.04,2.84,111.30,19.36
4,Albert,Johansson,5.06,3.29,126.64,22.47
...,...,...,...,...,...,...
194,Will,Wilson,5.06,3.05,115.97,19.96
195,William,Hackman,4.78,3.14,129.07,20.06
196,Woody,Hoffman,4.58,2.99,120.84,19.70
197,Woody,Jolie,4.35,2.60,114.94,19.41


## 2. Calcule a **soma** de amount (preço total do aluguel) por nome, sobrenome e email do cliente (customer) e indique os 10 clientes que mais gastaram e os 10 que menos gastaram:

In [3]:
conn = pg2.connect(host='localhost', 
                   port='5432', 
                   dbname='dvdrental', 
                   user='postgres', 
                   password='123')
cur = conn.cursor()

query = '''
        SELECT first_name, 
            last_name, 
            email, 
            SUM(amount) AS sum_amount
        FROM payment
        LEFT JOIN customer
            ON payment.customer_id = customer.customer_id
        GROUP BY first_name, last_name, email
        ORDER BY sum_amount;
        '''
cur.execute(query)
conn.commit()

data = cur.fetchall()

cur.close()
conn.close()

col_names = [desc.name for desc in cur.description]
df = pd.DataFrame(data=data, columns=col_names)

> **SQL Query (10 clientes que `mais` gastaram):**

```sql
SELECT first_name, 
	last_name, 
	email, 
	SUM(amount) AS sum_amount
FROM payment
LEFT JOIN customer
	ON payment.customer_id = customer.customer_id
GROUP BY first_name, last_name, email
ORDER BY sum_amount DESC
LIMIT 10;
```

In [4]:
print('10 clientes que MAIS gastaram:\n')

df.sort_values(by='sum_amount', ascending=False).head(n=10)

10 clientes que MAIS gastaram:



,first_name,last_name,email,sum_amount
598,Eleanor,Hunt,eleanor.hunt@sakilacustomer.org,211.55
597,Karl,Seal,karl.seal@sakilacustomer.org,208.58
596,Marion,Snyder,marion.snyder@sakilacustomer.org,194.61
595,Rhonda,Kennedy,rhonda.kennedy@sakilacustomer.org,191.62
594,Clara,Shaw,clara.shaw@sakilacustomer.org,189.60
593,Tommy,Collazo,tommy.collazo@sakilacustomer.org,183.63
592,Ana,Bradley,ana.bradley@sakilacustomer.org,167.67
591,Curtis,Irby,curtis.irby@sakilacustomer.org,167.62
590,Marcia,Dean,marcia.dean@sakilacustomer.org,166.61
589,Mike,Way,mike.way@sakilacustomer.org,162.67


> **SQL Query (10 clientes que `menos` gastaram):**

```sql
SELECT first_name, 
	last_name, 
	email, 
	SUM(amount) AS sum_amount
FROM payment
LEFT JOIN customer
	ON payment.customer_id = customer.customer_id
GROUP BY first_name, last_name, email
ORDER BY sum_amount ASC
LIMIT 10;
```

In [5]:
print('10 clientes que MENOS gastaram:\n')

df.sort_values(by='sum_amount', ascending=True).head(n=10)

10 clientes que MENOS gastaram:



,first_name,last_name,email,sum_amount
0,Brian,Wyman,brian.wyman@sakilacustomer.org,27.93
1,Leona,Obrien,leona.obrien@sakilacustomer.org,32.90
2,Caroline,Bowman,caroline.bowman@sakilacustomer.org,37.87
3,Anthony,Schwab,anthony.schwab@sakilacustomer.org,47.85
4,Tiffany,Jordan,tiffany.jordan@sakilacustomer.org,49.88
5,Kirk,Stclair,kirk.stclair@sakilacustomer.org,50.83
6,Bobbie,Craig,bobbie.craig@sakilacustomer.org,52.81
7,Jo,Fowler,jo.fowler@sakilacustomer.org,54.85
8,Penny,Neal,penny.neal@sakilacustomer.org,56.84
9,Johnny,Turpin,johnny.turpin@sakilacustomer.org,57.81


---